In [ ]:
# default_exp functional

In [ ]:
#export
from kesscore.imports import *
from functools import wraps

In [ ]:
#export
def _listify(f=None):
    if f==None: return lisitify_decorator
    @wraps(f)
    def _inner(*args, **kwargs): return list(f(*args, **kwargs))
    return _inner    

In [ ]:
#export
lmap = _listify(map)
lzip = _listify(zip)
lfilter = _listify(filter)

In [ ]:
test_eq_type(lmap(noop, [1,2,3]), [1,2,3])
test_eq_type(lzip([1,2],[3,4]),[(1,3),(2,4)])
test_eq_type(lfilter(ge(2), [1,2,3,4]), [2,3,4])

In [ ]:
#export
def _all(self:L)->bool:return all(self)
L.all = _all

In [ ]:
assert not L([True, False, True] ).all()
assert not L([True, True,  False]).all()
assert     L([True, True,  True] ).all()

In [ ]:
from nbdev.sync import notebook2script

In [ ]:
notebook2script()

Converted 00_functional.ipynb.
Converted 01_images.ipynb.
Converted 02_fastdl.ipynb.
Converted 03_datadownload.ipynb.
Converted index.ipynb.
